In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import requests
from PIL import Image
from io import BytesIO
from colorthief import ColorThief
import matplotlib.pyplot as plt

In [5]:
import pandas as pd 
import numpy as np 
import random 

# Data

The imformation is extracted from The Movie DB from https://www.themoviedb.org/ using API key

In [10]:
api_key = "3039136bdade60a401af3af4c8024a39"  

url = "https://api.themoviedb.org/3/authentication/token/new"

params = {
    "api_key": api_key
}

headers = {
    "accept": "application/json"
}

response = requests.get(url, headers=headers, params=params)

print(response.text)

{"success":true,"expires_at":"2025-06-04 19:49:01 UTC","request_token":"b4af815938fb0e988209e76f1672e42b770d79d0"}


In [16]:
movie_names = ['Avatar', 'Inception', 'Chicago']  # List of movie titles

def get_movie_info(movie_name):
    search_url = 'https://api.themoviedb.org/3/search/movie'
    params = {'api_key': api_key, 'query': movie_name}
    
    response = requests.get(search_url, params=params)
    data = response.json()
    
    if data['results']:
        movie = data['results'][0]
        movie_id = movie['id']
        
        details_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
        details_params = {'api_key': api_key}
        details_response = requests.get(details_url, params=details_params)
        details = details_response.json()
        
        title = details.get('title')
        description = details.get('overview')
        poster_path = details.get('poster_path')
        poster_url = f'https://image.tmdb.org/t/p/w500{poster_path}' if poster_path else 'No poster available'
        
        return {'title': title, 'description': description, 'poster_url': poster_url}
    else:
        return {'title': movie_name, 'description': 'Not found', 'poster_url': None}

for movie_name in movie_names:
    info = get_movie_info(movie_name)
    print(f"Title: {info['title']}")
    print(f"Description: {info['description']}")
    print(f"Poster URL: {info['poster_url']}")
    print('---')


Title: Avatar
Description: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
Poster URL: https://image.tmdb.org/t/p/w500/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
---
Title: Inception
Description: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.
Poster URL: https://image.tmdb.org/t/p/w500/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
---
Title: Chicago
Description: Based on a true story, two-timing boozing wife Roxie Hart kills her lover in cold blood after he leaves her, and finagles her way out being indicted. The basis for Kander/Ebb's 1975 Broadway musical of the same name and its Oscar-winning 2002 film adaptation.
Poster URL: https://image.tmdb.org/t/p

In [18]:
top_movies = []
page = 1

while len(top_movies) < 30:
    url = 'https://api.themoviedb.org/3/movie/popular'
    params = {
        'api_key': api_key,
        'page': page
    }
    response = requests.get(url, params=params)
    data = response.json()

    results = data.get('results', [])
    top_movies.extend(results)
    page += 1

# Trim to top 30
top_30 = top_movies[:30]

# Prepare list of dicts for DataFrame
movies_data = []
for movie in top_30:
    movies_data.append({
        'title': movie.get('title'),
        'description': movie.get('overview'),
        'rating': movie.get('vote_average'),
        'poster_url': f"https://image.tmdb.org/t/p/w500{movie.get('poster_path')}" if movie.get('poster_path') else None
    })

# Create DataFrame
df = pd.DataFrame(movies_data)

print(df)


                                        title  \
0                               Lilo & Stitch   
1                                     Sinners   
2                           A Minecraft Movie   
3                                    Sikandar   
4                               A Working Man   
5                Final Destination Bloodlines   
6                                Shadow Force   
7   Mission: Impossible - The Final Reckoning   
8                The Last Stand of Ellen Cole   
9                                  Until Dawn   
10                                    Warfare   
11                        The Ugly Stepsister   
12           Captain America: Brave New World   
13                              Lilo & Stitch   
14                            The Containment   
15                                 Snow White   
16                               Mountainhead   
17                             A Widow's Game   
18                                Tin Soldier   
19                  

In [34]:
df

,title,description,rating,poster_url
0,Lilo & Stitch,The wildly funny and touching story of a lonel...,7.098,https://image.tmdb.org/t/p/w500/tUae3mefrDVTgm...
1,Sinners,"Trying to leave their troubled lives behind, t...",7.505,https://image.tmdb.org/t/p/w500/yqsCU5XOP2mkbF...
2,A Minecraft Movie,Four misfits find themselves struggling with o...,6.485,https://image.tmdb.org/t/p/w500/yFHHfHcUgGAxzi...
3,Sikandar,A tragic accident pushes the powerful Sikandar...,5.220,https://image.tmdb.org/t/p/w500/t48miSSfe7COqg...
4,A Working Man,Levon Cade left behind a decorated military ca...,6.700,https://image.tmdb.org/t/p/w500/6FRFIogh3zFnVW...
5,Final Destination Bloodlines,"Plagued by a violent recurring nightmare, coll...",6.979,https://image.tmdb.org/t/p/w500/6WxhEvFsauuACf...
6,Shadow Force,Kyrah and Isaac were once the leaders of a mul...,6.100,https://image.tmdb.org/t/p/w500/uX6FaNE86a4Xnf...
7,Mission: Impossible - The Final Reckoning,Ethan Hunt and team continue their search for ...,7.135,https://image.tmdb.org/t/p/w500/z53D72EAOxGRqd...
8,The Last Stand of Ellen Cole,When an evil contractor goes to extreme measur...,8.036,https://image.tmdb.org/t/p/w500/oq1pGVQ2t3Cy4v...
9,Until Dawn,One year after her sister Melanie mysteriously...,6.485,https://image.tmdb.org/t/p/w500/juA4IWO52Fecx8...


In [60]:
top_movies = []
page = 1

while len(top_movies) < 1000:
    url = 'https://api.themoviedb.org/3/movie/top_rated'
    params = {
        'api_key': api_key,
        'page': page
    }
    response = requests.get(url, params=params)
    data = response.json()

    results = data.get('results', [])
    top_movies.extend(results)
    page += 1

In [62]:
top_movies

[{'adult': False,
  'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg',
  'genre_ids': [18, 80],
  'id': 278,
  'original_language': 'en',
  'original_title': 'The Shawshank Redemption',
  'overview': 'Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.',
  'popularity': 29.8924,
  'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg',
  'release_date': '1994-09-23',
  'title': 'The Shawshank Redemption',
  'video': False,
  'vote_average': 8.7,
  'vote_count': 28387},
 {'adult': False,
  'backdrop_path': '/tmU7GeKVybMWFButWEGl2M4GeiP.jpg',
  'genre_ids': [18, 80],
  'id': 238,
  'original_language': 'en',
  'original_title': 'The Godfather',
  'overview

In [66]:
movies_data = []
for movie in top_movies:
    movies_data.append({
        'title': movie.get('title'),
        'description': movie.get('overview'),
        'rating': movie.get('vote_average'),
        'poster_url': f"https://image.tmdb.org/t/p/w500{movie.get('poster_path')}" if movie.get('poster_path') else None
    })

# Create DataFrame
df = pd.DataFrame(movies_data)

df

,title,description,rating,poster_url
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,8.700,https://image.tmdb.org/t/p/w500/9cqNxx0GxF0bfl...
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",8.686,https://image.tmdb.org/t/p/w500/3bhkrj58Vtu7en...
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,8.570,https://image.tmdb.org/t/p/w500/hek3koDUyRQk7F...
3,Schindler's List,The true story of how businessman Oskar Schind...,8.564,https://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUY...
4,12 Angry Men,The defense and the prosecution have rested an...,8.548,https://image.tmdb.org/t/p/w500/ow3wq89wM8qd5X...
...,...,...,...,...
995,The Anthem of the Heart,A young girl had her voice magically taken awa...,7.600,https://image.tmdb.org/t/p/w500/7bJHM6t6j3YLjD...
996,The Little Prince,Based on the best-seller book 'The Little Prin...,7.609,https://image.tmdb.org/t/p/w500/je5Z7gbFTzrs3F...
997,Tangled,"Feisty teenager Rapunzel, who has long and mag...",7.609,https://image.tmdb.org/t/p/w500/ym7Kst6a4uodry...
998,Lilya 4-ever,"Lilya Michailova lives in poverty, dreaming of...",7.600,https://image.tmdb.org/t/p/w500/m1xeLa00P3her9...


In [68]:
df.to_csv('top_rated_movies.csv', index=False)